# **Data analysis on cancellation notebook**

## Objectives

* Answer business requirement, which is to answer the following questions from stakeholders:
    - Which months have the highest number of cancellations?
    - What are the top 5 countries with the highest number of cancellations?
    - Which booking channels have the highest number of cancellations?
    - Are bookings with weekend nights stay more likely to be cancelled than those with none?

## Inputs

* outputs/datasets/collections/hotel_bookings.csv

## Outputs

* Generate code to answer the above business requirement and use it to build the Streamlit app


---

# Change working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load data

In [ ]:
import pandas as pd

df = pd.read_csv("outputs/datasets/collection/hotel_bookings.csv")
df.head()

---

# Data exploration

Get familiar with the dataset, check variable type and distribution, missing levels, and what these variables mean in a business context.

In [ ]:
from ydata_profiling import ProfileReport

pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

# Conventional data analysis

Based on the above business requirement, the following variables will be studied to answer the questions from stakeholders:
- `arrival_date_month`
- `country`
- `distribution_channel`
- `stays_in_weekend_nights`

In [ ]:
vars_to_study = ['arrival_date_month', 'country', 'distribution_channel', 'stays_in_weekend_nights']
vars_to_study

In [ ]:
df_eda = df.filter(vars_to_study + ['is_canceled'])
df_eda.head(3)

## Arrival date month

The following question will be investigated:
- Which months have the highest number of cancellations?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Calculate total number of cancellations for each month
cancellations = df_eda[df_eda['is_canceled'] == 1].groupby(
    'arrival_date_month'
).size()

# Sort months by number of cancellations in descending order
sorted_months = cancellations.sort_values(ascending=False).index.tolist()

# Countplot
plt.figure(figsize=(12, 6))
sns.countplot(
    data=df_eda,
    x='arrival_date_month',
    hue='is_canceled',
    order=sorted_months
)
plt.xticks(rotation=90)
plt.title('Number of Cancellations per Month')
plt.xlabel('arrival_date_month')
plt.ylabel('count')
plt.show()

### Conclusion
August and July are the top 2 months with the highest number of cancellations. This is during the summer season.

## Country

The following question will be investigated:
- What are the top 5 countries with the highest number of cancellations?

In [ ]:
# Calculate total number of cancellations for each country
cancellations_by_country = df_eda[df_eda['is_canceled'] == 1].groupby(
    'country'
).size()

# Get the top 5 countries with the highest number of cancellations
top_5_countries = cancellations_by_country.nlargest(5).index.tolist()

# Filter the top 5 countries
df_top_5 = df_eda[df_eda['country'].isin(top_5_countries)]

# Countplot
plt.figure(figsize=(12, 6))
sns.countplot(
    data=df_top_5,
    x='country',
    hue='is_canceled',
    order=top_5_countries
)
plt.title('Top 5 Countries with the Highest Number of Cancellations')
plt.xlabel('country')
plt.ylabel('count')
plt.xticks(rotation=90)
plt.show()

### Conclusion
The top 5 countries with the highest number of cancellations are:
- Portugal
- The UK
- France
- Spain
- Italy

## Distribution channels

The following question will be investigated:
- Which booking channels have the highest number of cancellations?

In [ ]:
# Calculate total number of cancellations for each distribution channel
cancellations_by_channel = df_eda[df_eda['is_canceled'] == 1].groupby(
    'distribution_channel'
).size()

# Sort distribution channels by number of cancellations in descending order
sorted_channels = cancellations_by_channel.sort_values(
    ascending=False
).index.tolist()

# Countplot
plt.figure(figsize=(12, 6))
sns.countplot(
    data=df_eda,
    x='distribution_channel',
    hue='is_canceled',
    order=sorted_channels
)
plt.title('Number of Cancellations by Distribution Channel')
plt.xlabel('distribution_channel')
plt.ylabel('count')
plt.xticks(rotation=90)
plt.show()

### Conclusion
There are more cancellations of bookings made through TA's (Travel Agencies) or TO's (Tour Operators) as opposed to bookings made directly with the hotel.

## Weekend nights stay

The following question will be investigated:
- Are bookings with weekend nights stay more likely to be cancelled than those with none?

In [ ]:
# Histplot
plt.figure(figsize=(12, 6))
sns.histplot(
    data=df_eda,
    x='stays_in_weekend_nights',
    hue='is_canceled',
    kde=True,
    element='step'
)
plt.title('Distribution of Stays in Weekend Nights')
plt.xlim(0, 8)
plt.xlabel('stays_in_weekend_nights')
plt.ylabel('count')
plt.show()

### Conclusion
Bookings without any weekend nights stay are cancelled more than bookings with weekend nights stay.

---

# Summary of conclusions

- The following variables were studied to answer questions from stakeholders as per business requirement:
    - `arrival_date_month`
    - `country`
    - `distribution_channel`
    - `stays_in_weekend_nights`
- Conventional data analysis provided the following answers:
    - August and July are the top 2 months with the highest number of cancellations. This is during the summer season.
    - The top 5 countries with the highest number of cancellations are:
        - Portugal
        - The UK
        - France
        - Spain
        - Italy
    - There are more cancellations of bookings made through TA's (Travel Agencies) or TO's (Tour Operators) as opposed to bookings made directly with the hotel.
    - Bookings without any weekend nights stay are cancelled more than bookings with weekend nights stay.

---

# Next steps

Data cleaning will be conducted in preparation for modelling.